# Calcite

# lex: JAVA
* [Introduction to Apache Calcite](https://www.baeldung.com/apache-calcite)

In [1]:
public class Employee {
    public String name;
    public String id;

    public String deptId;

    public Employee(String name, String id, String deptId) {
        this.name = name;
        this.id = id;
        this.deptId = deptId;
    }
}

public class Department {
    public String deptId;
    public String deptName;

    public Department(String deptId, String deptName) {
        this.deptId = deptId;
        this.deptName = deptName;
    }
}

public class CompanySchema {
    public Employee[] employees;
    public Department[] departments;
}

CompanySchema companySchema = new CompanySchema();
companySchema.employees = new Employee[]{new Employee("e1", "e1", "d1")};
companySchema.departments = new Department[]{new Department("d1", "d1")};

[LREPL.$JShell$14$Department;@14c07b8c

In [1]:
%maven de.vandermeer:asciitable:0.3.2

import de.vandermeer.asciitable.*;
import java.util.*;

public class Asciitables {

    public static void printTable(List<String> header, List<List<String>> rows) {
        AsciiTable at = new AsciiTable();
        at.addRule();
        at.addRow(header);
        at.addRule();
        if (rows.size() > 0) {
            for (List<String> row : rows) {
                at.addRow(row);
            }
        }

        at.addRule();

        // http://www.vandermeer.de/projects/skb/java/asciitable/examples/AT_07d_LongestWord.html
        at.getRenderer().setCWC(new CWC_LongestWord());
        System.out.println(at.render());
    }
}

In [6]:

import java.sql.*;
import java.util.*;

// Parse ResultSet
public class SQLs {

    public static void parseRs(ResultSet rs) throws SQLException {
        List<String> header = new ArrayList<>();
        List<List<String>> rows = new ArrayList<>();

        ResultSetMetaData rsMeta = rs.getMetaData();
        int n = rsMeta.getColumnCount();
        for (int i = 1; i <= n; i++) {
            header.add(rsMeta.getColumnName(i));
        }
        while (rs.next()) {

            if (n > 0) {
                List<String> row = new ArrayList<>();
                for (int i = 1; i <= n; i++) {
                    String v = rs.getString(i);
                    if (Objects.isNull(v)) {
                        v = "";
                    }
                    row.add(v);
                }
                rows.add(row);
            }
        }

        Asciitables.printTable(header, rows);
    }
}

In [ ]:
%maven org.apache.calcite:calcite-core:1.38.0

import java.sql.*;

import org.apache.calcite.jdbc.CalciteConnection;
import org.apache.calcite.schema.SchemaPlus;
import org.apache.calcite.schema.Schema;

import org.apache.calcite.adapter.java.ReflectiveSchema;

public class Calcites {

    public static Connection getConnection(String url) {
        Connection connection = DriverManager.getConnection(url);
        DatabaseMetaData dbMeta = connection.getMetaData();
        System.out.println(dbMeta.getDriverName());
        return connection;
    }

    public static void getTables(Connection connection) {
        ResultSet rs = connection.getMetaData().getTables(null, null, null, null);

        SQLs.parseRs(rs);
    }

    public static void closeConnection(Connection connection) {
        connection.close();
    }
}


In [ ]:

String connUrl = "jdbc:calcite:lex=JAVA";
Connection connection = Calcites.getConnection(connUrl);
CalciteConnection calciteConnection = connection.unwrap(CalciteConnection.class);
SchemaPlus rootSchema = calciteConnection.getRootSchema();
Schema schema = new ReflectiveSchema(companySchema);
rootSchema.add("company", schema);

Calcites.getTables(connection)

// query
PreparedStatement ps = connection.prepareStatement(
"select dept.deptName, count(emp.id) " +
"from company.employees as emp " +
"  join company.departments as dept " + 
"    on (emp.deptId = dept.deptId) " +
"group by dept.deptName);"
ResultSet rs = ps.executeQuery();
SQLs.parseRs(rs);